In [1]:
import json, vincenty

## We live on a sphere, and that makes this hard

Given a bunch of points, compute some distances with shapely and pyproj

For simplicity, we keep all of our points in WGS84, we can use vincenty distance for super accurate results

In [6]:
# Let's load a user and get their points in WGS84

In [23]:
points = []
with open('/data/chime/geo2/NYC/ZoneA/alex_tonk.geojsonl') as inFile:
    for idx, line in enumerate(inFile):
        t = json.loads(line)
        points.append(t['geometry']['coordinates'])

points is now an array of points in lon, lat (x,y) format

In [24]:
points[:2]

[[-73.96226984, 40.78033596], [-73.96387997, 40.77846793]]

Use Vincenty Distance.

NOTE: To use Vincenty distance, the format is: (lat1, lon1, lat2, lon2), so reverse the GeoJSON representation of (x,y)

In [25]:
for p1 in points[:3]:
    for p2 in points[:3]:
        distance = vincenty.vincenty(list(reversed(p1)), list(reversed(p2)))
        print("Distance between {0} and {1}: {2} kilometers".format(p1,p2,distance))

Distance between [-73.96226984, 40.78033596] and [-73.96226984, 40.78033596]: 0.0 kilometers
Distance between [-73.96226984, 40.78033596] and [-73.96387997, 40.77846793]: 0.248006 kilometers
Distance between [-73.96226984, 40.78033596] and [-73.81858043, 40.75890912]: 12.362479 kilometers
Distance between [-73.96387997, 40.77846793] and [-73.96226984, 40.78033596]: 0.248006 kilometers
Distance between [-73.96387997, 40.77846793] and [-73.96387997, 40.77846793]: 0.0 kilometers
Distance between [-73.96387997, 40.77846793] and [-73.81858043, 40.75890912]: 12.458236 kilometers
Distance between [-73.81858043, 40.75890912] and [-73.96226984, 40.78033596]: 12.362479 kilometers
Distance between [-73.81858043, 40.75890912] and [-73.96387997, 40.77846793]: 12.458236 kilometers
Distance between [-73.81858043, 40.75890912] and [-73.81858043, 40.75890912]: 0.0 kilometers


The following link confirms that Google agrees with this distance: 

In [26]:
feats = []
for p in points[:3]: 
    feats.append({"type":"Feature","properties":{},"geometry":{"type":"Point", "coordinates":[p[0],p[1]]}})
print("http://townsendjennings.com/geo/?geojson="+json.dumps({"type":"FeatureCollection","features":feats}).replace(" ",""))

http://townsendjennings.com/geo/?geojson={"type":"FeatureCollection","features":[{"geometry":{"coordinates":[-73.96226984,40.78033596],"type":"Point"},"properties":{},"type":"Feature"},{"geometry":{"coordinates":[-73.96387997,40.77846793],"type":"Point"},"properties":{},"type":"Feature"},{"geometry":{"coordinates":[-73.81858043,40.75890912],"type":"Point"},"properties":{},"type":"Feature"}]}


Geometric Centorid?

In [27]:
len(points)

27

In [29]:
sum(p[0] for p in points)/len(points), sum(p[1] for p in points)/len(points)

(-73.95001657629629, 40.73189305185184)

In [43]:
from shapely import geometry

In [44]:
pts = geometry.MultiPoint([geometry.Point(x) for x in points])

In [48]:
print(pts.centroid)

POINT (-73.95001657629629 40.73189305185184)


Now add some units?

Eh... that's probably ok'